In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import pickle as pk
import os
print(os.listdir("../input"))
from PIL import Image
import matplotlib.pyplot as plt
# Any results you write to the current directory are saved as output.

from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)


In [ ]:
PATH = "../input/train"

In [ ]:
im_array = []
for filename in os.listdir(PATH)[:10000]:
    im = Image.open(os.path.join(PATH,filename)) #grayscale
    im = np.array(im)
    im_array.append(np.array(im).reshape(3, im.shape[0], im.shape[1]))
    
    


In [ ]:
im_array_2 = np.array(im_array).reshape(len(im_array), 96,96,3)
im_array_2 = (im_array_2 - im_array_2.mean()) / im_array_2.std()
im_array_2[0].shape

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, UpSampling2D, Dense
from keras.models import Model
from keras.optimizers import Adam, SGD
act = "tanh"
inputs = Input(shape=(96,96,3))
x = Conv2D(16, kernel_size = (3,3),padding = "same", activation = act)(inputs)

x = MaxPooling2D(pool_size=(2,2))(x)
x = Conv2D(64, kernel_size = (3,3), padding= "same", activation = act)(x)

code = MaxPooling2D(pool_size=(2,2))(x)
x = Conv2D(128, kernel_size=(3,3), padding = "same", activation = act)(code)

x = UpSampling2D((2,2))(x)
x = Conv2D(64, kernel_size = (3,3), padding = "same", activation = act)(x)

x = UpSampling2D((2,2))(x)
x = Conv2D(16, kernel_size=(3,3),padding = "same", activation = act)(x)

x = Conv2D(3, kernel_size =(3,3), padding = "same", activation = act)(x)
cnn_autoencoder = Model(inputs,x)


cnn_autoencoder.summary()
optimizer = SGD(lr = 0.1, momentum=0.7)
cnn_autoencoder.compile(optimizer=optimizer, loss='mse', metrics = ['accuracy'])

In [ ]:
history = cnn_autoencoder.fit(im_array_2,im_array_2, epochs = 10, validation_split = 0.1)

In [ ]:
plt.plot(history.history['loss'])

In [ ]:
predictions = cnn_autoencoder.predict(im_array_2)

In [ ]:
fig, (ax1, ax2)  = plt.subplots(1,2)
ax1.imshow(im_array_2[10])
ax1.set_title("Original")
ax2.imshow(predictions[10])
ax2.set_title("Predicted")

In [ ]:
train_labels_df = pd.read_csv("../input/train_labels.csv").iloc[:10000,1]
train_labels_df.head()